<a href="https://colab.research.google.com/github/YEONGHUN-H/YEONGHUN-H/blob/main/%EC%A3%BC%EA%B0%80%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

In [2]:
pip install yfinance pandas numpy sklearn tensorflow

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta

class StockPredictor:
    def __init__(self, ticker_symbol, prediction_days=30):
        self.ticker = ticker_symbol
        self.prediction_days = prediction_days
        self.scaler = MinMaxScaler()

    def prepare_data(self, start_date='2024-10-01'):
        """데이터 수집 및 전처리"""
        # 주가 데이터 수집
        stock = yf.Ticker(self.ticker)
        df = stock.history(start=start_date)

        # 기술적 지표 추가
        df['MA5'] = df['Close'].rolling(window=5).mean()
        df['MA20'] = df['Close'].rolling(window=20).mean()
        df['MA60'] = df['Close'].rolling(window=60).mean()

        # RSI 계산
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # MACD 계산
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

        # 거래량 지표
        df['Volume_MA5'] = df['Volume'].rolling(window=5).mean()

        # 결측값 제거
        df.dropna(inplace=True)

        # 특성 선택
        features = ['Close', 'Volume', 'MA5', 'MA20', 'MA60', 'RSI', 'MACD', 'Signal_Line', 'Volume_MA5']
        self.feature_data = df[features]

        return df

    def create_sequences(self, data):
        """시계열 데이터를 시퀀스로 변환"""
        X, y = [], []
        for i in range(len(data) - self.prediction_days):
            X.append(data[i:(i + self.prediction_days)])
            y.append(data[i + self.prediction_days, 0])  # Close 가격만 예측
        return np.array(X), np.array(y)

    def build_model(self, input_shape):
        """LSTM 모델 구축"""
        model = Sequential([
            LSTM(units=50, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(units=50, return_sequences=True),
            Dropout(0.2),
            LSTM(units=50),
            Dropout(0.2),
            Dense(units=1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def train_model(self):
        """모델 학습"""
        # 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.fit_transform(self.feature_data)

        # 시퀀스 생성
        X, y = self.create_sequences(scaled_data)

        # 학습/테스트 분할
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 모델 생성 및 학습
        model = self.build_model((self.prediction_days, X.shape[2]))
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

        # 예측 및 평가
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # 스케일 역변환
        train_predict = self.inverse_transform_predictions(train_predict)
        y_train_actual = self.inverse_transform_predictions(y_train.reshape(-1, 1))
        test_predict = self.inverse_transform_predictions(test_predict)
        y_test_actual = self.inverse_transform_predictions(y_test.reshape(-1, 1))

        # 성능 평가
        train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predict))
        test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predict))
        train_r2 = r2_score(y_train_actual, train_predict)
        test_r2 = r2_score(y_test_actual, test_predict)

        return {
            'model': model,
            'metrics': {
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_r2': train_r2,
                'test_r2': test_r2
            }
        }

    def inverse_transform_predictions(self, predictions):
        """예측값을 원래 스케일로 변환"""
        dummy = np.zeros((predictions.shape[0], self.feature_data.shape[1]))
        dummy[:, 0] = predictions[:, 0]
        return self.scaler.inverse_transform(dummy)[:, 0]

    def predict_next_days(self, model, days=7):
        """향후 n일 예측"""
        # 최근 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.transform(self.feature_data)
        last_sequence = scaled_data[-self.prediction_days:]

        predictions = []
        current_sequence = last_sequence.copy()

        # n일 동안 예측 수행
        for _ in range(days):
            # 다음 날 예측
            next_day = model.predict(current_sequence.reshape(1, self.prediction_days, scaled_data.shape[1]))
            predictions.append(next_day[0, 0])

            # 시퀀스 업데이트
            current_sequence = np.roll(current_sequence, -1, axis=0)
            current_sequence[-1] = next_day

        # 예측값을 원래 스케일로 변환
        predictions = np.array(predictions).reshape(-1, 1)
        return self.inverse_transform_predictions(predictions)

# 사용 예시
if __name__ == "__main__":
    # 삼성전자 주가 예측 모델 생성
    predictor = StockPredictor("005930.KS")
    # 카카오 주가 예측
    predictor_kakao = StockPredictor("035720.KS")
    results_kakao = predictor_kakao.train_model()
    # 현대차 주가 예측
    predictor_hyundai = StockPredictor("005380.KS")
    results_hyundai = predictor_hyundai.train_model()
    # 네이버 주가 예측
    predictor_naver = StockPredictor("035420.KS")
    results_naver = predictor_naver.train_model()

    # 모델 학습
    results = predictor.train_model()
    model = results['model']
    metrics = results['metrics']

    # 성능 지표 출력
    print("\n=== 모델 성능 ===")
    print(f"Training RMSE: {metrics['train_rmse']:.2f}")
    print(f"Test RMSE: {metrics['test_rmse']:.2f}")
    print(f"Training R²: {metrics['train_r2']:.4f}")
    print(f"Test R²: {metrics['test_r2']:.4f}")

    # 다음 7일 예측
    future_predictions = predictor.predict_next_days(model, days=7)
    print("\n=== 향후 7일 예측 가격 ===")
    for i, pred in enumerate(future_predictions, 1):
        print(f"{i}일 후 예상 가격: {pred:,.0f}원")

ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required by MinMaxScaler.

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta

class StockPredictor:
    def __init__(self, ticker_symbol, prediction_days=30):
        self.ticker = ticker_symbol
        self.prediction_days = prediction_days
        self.scaler = MinMaxScaler()

    def prepare_data(self):
        """데이터 수집 및 전처리"""
        # 주가 데이터 수집 (하루 전 데이터까지)
        stock = yf.Ticker(self.ticker)
        end_date = datetime.now() - timedelta(days=1)
        start_date = end_date - timedelta(days=180)  # 최근 6개월 데이터
        df = stock.history(start=start_date, end=end_date)

        # 기술적 지표 추가
        df['MA5'] = df['Close'].rolling(window=5).mean()
        df['MA20'] = df['Close'].rolling(window=20).mean()
        df['MA60'] = df['Close'].rolling(window=60).mean()

        # RSI 계산
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # MACD 계산
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

        # 거래량 지표
        df['Volume_MA5'] = df['Volume'].rolling(window=5).mean()

        # 결측값 제거
        df.dropna(inplace=True)

        # 특성 선택
        features = ['Close', 'Volume', 'MA5', 'MA20', 'MA60', 'RSI', 'MACD', 'Signal_Line', 'Volume_MA5']
        self.feature_data = df[features]

        return df

    def create_sequences(self, data):
        """시계열 데이터를 시퀀스로 변환"""
        X, y = [], []
        for i in range(len(data) - self.prediction_days):
            X.append(data[i:(i + self.prediction_days)])
            y.append(data[i + self.prediction_days, 0])  # Close 가격만 예측
        return np.array(X), np.array(y)

    def build_model(self, input_shape):
        """LSTM 모델 구축"""
        model = Sequential([
            LSTM(units=50, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(units=50, return_sequences=True),
            Dropout(0.2),
            LSTM(units=50),
            Dropout(0.2),
            Dense(units=1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def train_model(self):
        """모델 학습"""
        # 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.fit_transform(self.feature_data)

        # 시퀀스 생성
        X, y = self.create_sequences(scaled_data)

        # 학습/테스트 분할
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 모델 생성 및 학습
        model = self.build_model((self.prediction_days, X.shape[2]))
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

        # 예측 및 평가
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # 스케일 역변환
        train_predict = self.inverse_transform_predictions(train_predict)
        y_train_actual = self.inverse_transform_predictions(y_train.reshape(-1, 1))
        test_predict = self.inverse_transform_predictions(test_predict)
        y_test_actual = self.inverse_transform_predictions(y_test.reshape(-1, 1))

        # 성능 평가
        train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predict))
        test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predict))
        train_r2 = r2_score(y_train_actual, train_predict)
        test_r2 = r2_score(y_test_actual, test_predict)

        return {
            'model': model,
            'metrics': {
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_r2': train_r2,
                'test_r2': test_r2
            }
        }

    def inverse_transform_predictions(self, predictions):
        """예측값을 원래 스케일로 변환"""
        dummy = np.zeros((predictions.shape[0], self.feature_data.shape[1]))
        dummy[:, 0] = predictions[:, 0]
        return self.scaler.inverse_transform(dummy)[:, 0]

    def predict_next_days(self, model, days=7):
        """향후 n일 예측"""
        # 최근 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.transform(self.feature_data)
        last_sequence = scaled_data[-self.prediction_days:]

        predictions = []
        current_sequence = last_sequence.copy()

        # n일 동안 예측 수행
        for _ in range(days):
            # 다음 날 예측
            next_day = model.predict(current_sequence.reshape(1, self.prediction_days, scaled_data.shape[1]))
            predictions.append(next_day[0, 0])

            # 시퀀스 업데이트
            current_sequence = np.roll(current_sequence, -1, axis=0)
            current_sequence[-1] = next_day

        # 예측값을 원래 스케일로 변환
        predictions = np.array(predictions).reshape(-1, 1)
        return self.inverse_transform_predictions(predictions)

# 사용 예시
if __name__ == "__main__":
    # 삼성전자 주가 예측 모델 생성
    predictor = StockPredictor("005930.KS")

    # 모델 학습
    results = predictor.train_model()
    model = results['model']
    metrics = results['metrics']

    # 성능 지표 출력
    print("\n=== 모델 성능 ===")
    print(f"Training RMSE: {metrics['train_rmse']:.2f}")
    print(f"Test RMSE: {metrics['test_rmse']:.2f}")
    print(f"Training R²: {metrics['train_r2']:.4f}")
    print(f"Test R²: {metrics['test_r2']:.4f}")

    # 다음 7일 예측
    future_predictions = predictor.predict_next_days(model, days=7)
    print("\n=== 향후 7일 예측 가격 ===")
    for i, pred in enumerate(future_predictions, 1):
        print(f"{i}일 후 예상 가격: {pred:,.0f}원")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - loss: 0.0751 - val_loss: 0.0123
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - loss: 0.0240 - val_loss: 0.0036
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - loss: 0.0071 - val_loss: 0.0162
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - loss: 0.0189 - val_loss: 0.0179
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step - loss: 0.0183 - val_loss: 0.0110
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - loss: 0.0113 - val_loss: 0.0044
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step - loss: 0.0071 - val_loss: 0.0016
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - loss: 0.0039 - val_loss: 0.0021
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 359ms/step - loss: 0.0034 - val_loss: 0.0040
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - loss: 0.0079 - val_loss: 0.0054
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step - loss: 0.0090 - val_loss: 0.0057
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0114 - val_loss: 0.0050
Epoc

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from datetime import datetime, timedelta
from tensorflow.keras.layers import Input

class StockPredictor:
    def __init__(self, ticker_symbol, prediction_days=30):
        self.ticker = ticker_symbol
        self.prediction_days = prediction_days
        self.scaler = MinMaxScaler()

    def prepare_data(self):
        """데이터 수집 및 전처리"""
        # 주가 데이터 수집 (하루 전 날짜까지)
        stock = yf.Ticker(self.ticker)

        # 날짜 설정부분!!!!!!!!!!!!!!!
        end_date = datetime.now() - timedelta(days=0)  # 오늘로부터 하루 전 날짜까지 -> 현재 날짜로부터 언제까지 가져오겠다를 해야해서 end를 먼저 써야함
        start_date = end_date - timedelta(days=180)    # 하루 전부터 12개월 전까지 데이터부터
        print(f"Start Date: {start_date.strftime('%Y-%m-%d')}, End Date: {end_date.strftime('%Y-%m-%d')}")  # 날짜 숫자로 출력

        # 주가 데이터 가져오기
        df = stock.history(start=start_date, end=end_date)

        # 기술적 지표 추가
        df['MA5'] = df['Close'].rolling(window=5).mean()
        df['MA20'] = df['Close'].rolling(window=20).mean()
        df['MA60'] = df['Close'].rolling(window=60).mean()

        # RSI 계산
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))

        # MACD 계산
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

        # 거래량 지표
        df['Volume_MA5'] = df['Volume'].rolling(window=5).mean()

        # 결측값 중앙값으로
        df.dropna(inplace=True)

        # 특성 선택
        features = ['Close', 'Volume', 'MA5', 'MA20', 'MA60', 'RSI', 'MACD', 'Signal_Line', 'Volume_MA5']
        self.feature_data = df[features]

        return df

    def create_sequences(self, data):
        """시계열 데이터를 시퀀스로 변환"""
        X, y = [], []
        for i in range(len(data) - self.prediction_days):
            X.append(data[i:(i + self.prediction_days)])
            y.append(data[i + self.prediction_days, 0])  # Close 가격만 예측
        return np.array(X), np.array(y)

    def build_model(self, input_shape):
        """LSTM 모델 구축"""
        model = Sequential([
            Input(shape=input_shape),  # 첫 번째 레이어로 Input 사용
            LSTM(units=50, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(units=50, return_sequences=True),
            Dropout(0.2),
            LSTM(units=50),
            Dropout(0.2),
            Dense(units=1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    def train_model(self):
        """모델 학습"""
        # 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.fit_transform(self.feature_data)

        # 시퀀스 생성
        X, y = self.create_sequences(scaled_data)

        # 학습/테스트 분할
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 모델 생성 및 학습
        model = self.build_model((self.prediction_days, X.shape[2]))
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

        # 예측 및 평가
        train_predict = model.predict(X_train)
        test_predict = model.predict(X_test)

        # 스케일 역변환
        train_predict = self.inverse_transform_predictions(train_predict)
        y_train_actual = self.inverse_transform_predictions(y_train.reshape(-1, 1))
        test_predict = self.inverse_transform_predictions(test_predict)
        y_test_actual = self.inverse_transform_predictions(y_test.reshape(-1, 1))

        # 성능 평가
        train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predict))
        test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predict))
        train_r2 = r2_score(y_train_actual, train_predict)
        test_r2 = r2_score(y_test_actual, test_predict)

        return {
            'model': model,
            'metrics': {
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_r2': train_r2,
                'test_r2': test_r2
            }
        }

    def inverse_transform_predictions(self, predictions):
        """예측값을 원래 스케일로 변환"""
        dummy = np.zeros((predictions.shape[0], self.feature_data.shape[1]))
        dummy[:, 0] = predictions[:, 0]
        return self.scaler.inverse_transform(dummy)[:, 0]

    def predict_next_days(self, model, days=7):
        """향후 n일 예측"""
        # 최근 데이터 준비
        df = self.prepare_data()
        scaled_data = self.scaler.transform(self.feature_data)
        last_sequence = scaled_data[-self.prediction_days:]

        predictions = []
        current_sequence = last_sequence.copy()

        # n일 동안 예측 수행
        for _ in range(days):
            # 다음 날 예측
            next_day = model.predict(current_sequence.reshape(1, self.prediction_days, scaled_data.shape[1]))
            predictions.append(next_day[0, 0])

            # 시퀀스 업데이트
            current_sequence = np.roll(current_sequence, -1, axis=0)
            current_sequence[-1] = next_day

        # 예측값을 원래 스케일로 변환
        predictions = np.array(predictions).reshape(-1, 1)
        return self.inverse_transform_predictions(predictions)

# 사용 예시
if __name__ == "__main__":
    # 삼성전자 주가 예측 모델 생성
    predictor = StockPredictor("005930.KS")

    # 모델 학습
    results = predictor.train_model()
    model = results['model']
    metrics = results['metrics']

    # 성능 지표 출력
    print("\n=== 모델 성능 ===")
    print(f"Training RMSE: {metrics['train_rmse']:.2f}")
    print(f"Test RMSE: {metrics['test_rmse']:.2f}")
    print(f"Training R²: {metrics['train_r2']:.4f}")
    print(f"Test R²: {metrics['test_r2']:.4f}")

    # 다음 7일 예측
    future_predictions = predictor.predict_next_days(model, days=7)
    print("\n=== 향후 7일 예측 가격 ===")
    for i, pred in enumerate(future_predictions, 1):
        print(f"{i}일 후 예상 가격: {pred:,.0f}원")


Start Date: 2024-05-11, End Date: 2024-11-07


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.0430 - val_loss: 0.0042
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.0101 - val_loss: 0.0055
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0180 - val_loss: 0.0068
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.0199 - val_loss: 0.0019
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - loss: 0.0145 - val_loss: 3.5742e-04
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0054 - val_loss: 0.0031
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - loss: 0.0060 - val_loss: 0.0060
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - loss: 0.0089 - val_loss: 0.0065
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 0.0082 - val_loss: 0.0051
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.0098 - val_loss: 0.0027
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - loss: 0.0046 - val_loss: 0.0011
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - loss: 0.0025 - val_lo